In [14]:
import requests
from bs4 import BeautifulSoup
ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
headers = {'User-Agent': ua}

# 全体の流れ
- リスト`links1`にチラシを取得する店の階層1リンクを格納(例:https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470)
- 店ごとに取得する条件が異なるため、`link1_to_link2(link1)`という関数を作成。この関数は、リスト`links1`に含まれる適切なチラシの階層2リンク(string)のlist(`links2raw`)を返す
- `links2raw`には最新ではないチラシも含まれるため、関数`get_new_urls(links2raw)`を作成。`./urls.txt`を利用。返り値`links2`には新しいチラシの階層2リンク
- get_urls()で階層2のリンクを階層3のリンクに変換
- 内部にダウンロード

## リスト`links1`にチラシを取得する店の階層1リンクを格納(例:https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470)


### チラシ1枚式/シュフー
- ライフ北白川
- ライフ東一条(北白川と同じ)
- グレースたなか本店

### チラシ1枚式/生鮮舘
- 生鮮館白川
- 生鮮館里の前

### 単品/シュフー
- フレスコ北白川
- コレモ出町柳

In [1]:
links1 = []
links1.append("https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470") #ライフ
links1.append("https://tokubai.co.jp/%E3%82%B0%E3%83%AC%E3%83%BC%E3%82%B9%E3%81%9F%E3%81%AA%E3%81%8B/12265") #グレースたなか


階層構造

あるスーパーが出している全チラシ(階層1)

→1枚のチラシを表示するリンク(階層2)

→チラシの具体的なリンク(階層3)



https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470

→https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41074828?from=leaflet_navigation&origin_shop_id=9470

→https://image.tokubai.co.jp/images/bargain_office_leaflets/o=true/5858474.jpg?1682066150

In [2]:
# 公式サイトからチラシリンク一覧取得(元コード)

def get_urls():
  import requests
  from bs4 import BeautifulSoup

  url = "https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41074828?from=leaflet_navigation&origin_shop_id=9470"


  html = requests.get(url)
  soup = BeautifulSoup(html.text, 'html.parser')

  # ★以下は実際のHTML構成に合わせて実装する
  flyer_list = soup.find_all('table')
  url_list = []
  for flyer in flyer_list:
    # 日付
    date = flyer.find('div', {'class': 'sale'}).find('a').get_text(strip=True).replace(' ', '').replace('（', '(').replace('）', ')')

    # PDF
    url_info = {}
    url_info['date'] = date
    url_info['url'] = flyer.find('a', {'title': 'PDF'})['href']
    url_list.append(url_info)

  return url_list

In [ ]:
#エラー吐かれる
import requests
from bs4 import BeautifulSoup

url = "https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41074828?from=leaflet_navigation&origin_shop_id=9470"


html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')
print(soup)

In [3]:
#"https://qiita.com/chiguh28/items/b3bfcd6404caeb3b5dde" を参考に改良
ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
headers = {'User-Agent': ua}

r = requests.get(url,headers=headers)
soup = BeautifulSoup(r.text,"html.parser")
print(soup)

<!DOCTYPE html>

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>ライフ 北白川店のチラシ・店舗情報 | トクバイ</title><meta content="特売,トクバイ,セール,バーゲン,チラシ,クーポン,イベント,クチコミ,レシピ,簡単,料理,recipe" name="keywords"><meta content="スーパー・ドラッグストア掲載数No.1チラシサイト、「トクバイ」でチラシをチェック！クーポン・タイムセール・バーゲン情報で賢く節約！" name="description"><meta content="noindex,follow,noarchive" name="robots"/><meta content="nositelinkssearchbox" name="google"/>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="n+BgS25ZFFDNT/XZxzdTRlv5HQa9qwp6tEnFL75GlifKmdxc9iQ/q6nA519iSSkGX4IiaLD56nhFWWYea+85gQ==" name="csrf-token"/>
<link href="/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41074828?from=leaflet_navigation&amp;origin_shop_id=9470" hreflang="ja" rel="alternate"/>
<link href="https://assets.tokubai.co.jp/assets/themes/bargain/favicon-733d1fdfb411b42bb2ad33d23a6e73d50ececb743ebcf2051fd414d8b52d9994.ico" rel="icon"/>
<link href="https://assets.toku

In [22]:
#`soup`からチラシのurlを取得　参考：https://qiita.com/neet-AI/items/98d4194872ee4f53e3b4
flyer_list = soup.find_all("img",class_="leaflet")
print(flyer_list)
print(set(tag['src'] for tag in flyer_list))
print(soup.find('img',class_='leaflet').get('src'))

[<img alt="2023年4月22〜25日までのチラシ" class="leaflet transparent" src="https://image.tokubai.co.jp/images/bargain_office_leaflets/o=true/5858474.jpg?1682066150"/>]
{'https://image.tokubai.co.jp/images/bargain_office_leaflets/o=true/5858474.jpg?1682066150'}
https://image.tokubai.co.jp/images/bargain_office_leaflets/o=true/5858474.jpg?1682066150


In [23]:
#まとめ： 関数内のurlにあるチラシ.jpgのリンクを出力する`get_url()`
def get_url():
  import requests
  from bs4 import BeautifulSoup

  url = "https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41074828?from=leaflet_navigation&origin_shop_id=9470"
  ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
  headers = {'User-Agent': ua}

  r = requests.get(url,headers=headers)
  soup = BeautifulSoup(r.text,"html.parser")
  return soup.find('img',class_='leaflet').get('src')

In [24]:
get_url()

'https://image.tokubai.co.jp/images/bargain_office_leaflets/o=true/5858474.jpg?1682066150'

In [13]:
#これから作業の第一段階のテスト
url = "https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470"
r = requests.get(url,headers=headers)
soup = BeautifulSoup(r.text,"html.parser")
print(soup.find_all(class_="image_element")[0])
print("")
print(len(soup.find(class_="image_element").find(class_="description")))

NameError: name 'headers' is not defined

In [49]:
print(soup.find(class_="image_element").find(class_="description").string)


2023年4月22〜25日まで



In [45]:
print((soup.find(class_="image_element").find(class_="description").string)[1:])

2023年4月22〜25日まで



In [65]:
#チラシ一覧のサイトからタイトルとサブタイトルを抽出、ここから条件分岐を作れないだろうか？
elements = soup.find_all(class_="image_element")
for element in elements:
    date = element.find(class_="description").string[1:-1]
    print(date)
    title = element.find(class_="text hoverable_link").string[1:-1]
    print(title)
    link = element.get("href")
    print(link)
    print(url+link)

2023年4月22〜25日まで
くらし応援祭
/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41074828
https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41074828
2023年4月22〜25日まで
生鮮大市
/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41074678
https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41074678
2023年4月22〜25日まで
【Wｅｂ】エチケット・UV
/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41073859
https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41073859
2023年4月1〜30日まで
【WEB】BIORAL NEWS
/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/39939144
https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/39939144


In [3]:
old_url_list = []
with open('./urls.txt', 'r') as f:
    old_url_list = f.read().splitlines()



2

In [71]:
#urlが新しいものかどうか判定するやつを作る
#'./urls.txt'には過去に取得した階層2リンクが保存されていると仮定
#old_urlsとnew_urlsを比較、「oldにありnewにもある」リンクと「oldになくnewにはある」リンク一覧を`./urls.txt`に保存
#「oldになくnewにはある」リンクのリストが返り値

def get_new_urls(new_urls): #urlsの中には現在ある
  # urls.txt読込
  old_urls = []
  with open('/urls.txt', 'r') as f:
    old_urls = f.read().splitlines()

  new_urls = []
  urls_text = []
  count = 0
  for new_url in new_urls:
    urls_text.append(new_url + '\n')

    if new_url not in old_urls:
      # 新規
      new_url = count
      new_urls.append(new_url)
      count += 1

  # 今回のURL一覧をurls.txtに書込
  f = open('/urls.txt', 'w')
  f.writelines(urls_text)
  f.close()

  return new_urls

In [72]:
get_new_urls([url])

TypeError: 'str' object does not support item assignment

In [68]:
url

'https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470'